# Flavor Transformations

This notebook produces the figures in the SNEWPY paper showing the effect of the flavor transformation prescriptions upon the nakazato-shen-z0.004-t_rev100ms-s20.0 model 

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

from snewpy.neutrino import MassHierarchy, MixingParameters, ThreeFlavorMixingParameters, FourFlavorMixingParameters
import snewpy.flavor_transformation as xforms
from snewpy.flavor import ThreeFlavor, FourFlavor

from snewpy.models.ccsn import Nakazato_2013

from astropy import units as u
from astropy import constants as c
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.time import Time                                       

mpl.rc('font', size=18)
%matplotlib inline

In [ ]:
model = Nakazato_2013(progenitor_mass=20*u.solMass, revival_time=100*u.ms, metallicity=0.004, eos='shen')
model

## Transformed and untransformed energy integrated flux at Earth

Compute and plot the flux at Earth for a SN at 10 kpc with no flavor transformation, and with the chosen flavor transformation.

In [ ]:
def plot_total_flux(model, xform_nmo, xform_imo):
    """Plot initial and oscillated neutrino luminosities.
    
    Parameters
    ----------
    model : SupernovaModel
        An input model from a CCSN simulation.
    flav_xform : FlavorTransformation
        A FlavorTransformation subclass; used to create an instance.
    """
    
    energies = np.linspace(1,60,119) * u.MeV    
    d = (10*u.kpc).to('cm').value # distance to SN
        
    times = model.get_time()
    burst_epoch = times <= 0.1*u.s
    accretion_epoch = (times > 0.1*u.s) & (times <= 0.5*u.s)
    cooling_epoch = (times > 0.5*u.s) & (times <= 10*u.s)
    
    ispec = model.get_flux(times, energies, d)
    ospec_nmo = model.get_flux(times, energies, d, xform_nmo)
    ospec_imo = model.get_flux(times, energies, d, xform_imo)
    
    ilum = ispec.integrate_or_sum('energy') 
    olum_nmo = ospec_nmo.integrate_or_sum('energy') 
    olum_imo = ospec_imo.integrate_or_sum('energy') 
    
    # make the figures 
    fig, axes = plt.subplots(3,3, figsize=(20,12), tight_layout=True)
    
    smax = [0.,0.,0.]
    titles = ['Untransformed', 'Transformed (NMO)', 'Transformed (IMO)']
    for i, spec in enumerate([ilum, olum_nmo, olum_imo]):   
        for j, phase in enumerate([burst_epoch, accretion_epoch, cooling_epoch]):
            ax = axes[i,j]
            timeunits = 'ms' if j==0 else 's'
                
            for flavor in ThreeFlavor:
                if i == 0:
                    smax[j] = np.maximum(smax[j], 1.1*np.max(spec.array.squeeze()[flavor][phase]))
                    
                ax.plot(times[phase].to(timeunits),
                        spec.array.squeeze()[flavor][phase], label=flavor.to_tex(), lw=3,
                        color='C0' if flavor.is_electron else 'C1',
                        ls='-' if flavor.is_neutrino else ':')
            
            ax.set(xlim=(times[phase][0].to(timeunits).value, times[phase][-1].to(timeunits).value),
                   ylim=(0, smax[j].value))
            
            if j==0:
               ax.set(ylabel=r'flux [$10^{16}$ erg$^{-1}$ cm$^{-2}$ s$^{-1}$]')
               ax.legend(loc='upper right', ncol=1, fontsize=18)
            if j==1:
                ax.set(title=titles[i])
            if i < 2:
                ax.set(xticklabels=[])
            else:
                ax.set(xlabel='time [{}]'.format(timeunits))
            
            ax.grid(ls=':')

      
    return fig

## Untransformed and Transformed Spectra at Earth

Compute and plot the spectra at Earth for a SN at 10 kpc with no flavor transformation, and with the chosen flavor transformation.

In [ ]:
def plot_spectra(model, xform_nmo, xform_imo, t):
    """Plot initial and oscillated neutrino luminosities.
    
    Parameters
    ----------
    model : SupernovaModel
        An input model from a CCSN simulation.
    flav_xform : FlavorTransformation
        A FlavorTransformation subclass; used to create an instance.
    t : astropy.Quantity
        Time to compute the spectrum.
    """

    energies = np.linspace(1,60,119) * u.MeV   
    d = (10*u.kpc).to('cm').value # distance to SN

    #get the spectra
    ispec = model.get_flux(t, energies, d)
    ospec_nmo = model.get_flux(t, energies, d, xform_nmo)
    ospec_imo = model.get_flux(t, energies, d, xform_imo)
    
    fig, axes = plt.subplots(2,2, figsize=(18,15), sharex=True, sharey=True, tight_layout=True)

    for i, spec in enumerate([ispec, ospec_nmo]):
        axes[0][0].plot(energies, spec.array.squeeze()[ThreeFlavor.NU_E]/1e16, 
                    label='Untransformed '+ThreeFlavor.NU_E.to_tex() if i==0 else 'Transformed '+ThreeFlavor.NU_E.to_tex(),
                    color='C0', ls='-' if i==0 else ':', lw=2,  alpha=0.7)
        axes[0][0].plot(energies, spec.array.squeeze()[ThreeFlavor.NU_MU]/1e16, 
                    label='Untransformed '+ThreeFlavor.NU_MU.to_tex() if i==0 else 'Transformed '+ThreeFlavor.NU_MU.to_tex(),
                    color='C1', ls='-' if i==0 else ':', lw=2,  alpha=0.7)

        axes[0][0].set(title='Neutrinos in the NMO: $t = ${:.1f}'.format(t))
        axes[0][0].grid()
        axes[0][0].legend(loc='upper right', ncol=2, fontsize=16)
    
        axes[0][1].plot(energies, spec.array.squeeze()[ThreeFlavor.NU_E_BAR]/1e16, 
                    label='Untransformed '+ThreeFlavor.NU_E_BAR.to_tex() if i==0 else 'Transformed '+ThreeFlavor.NU_E_BAR.to_tex(),
                    color='C0', ls='-' if i==0 else ':', lw=2,  alpha=0.7)
        axes[0][1].plot(energies, spec.array.squeeze()[ThreeFlavor.NU_MU_BAR]/1e16, 
                    label='Untransformed '+ThreeFlavor.NU_MU_BAR.to_tex() if i==0 else 'Transformed '+ThreeFlavor.NU_MU_BAR.to_tex(),
                    color='C1', ls='-' if i==0 else ':', lw=2,  alpha=0.7)

        axes[0][1].set(title='Antineutrinos in the NMO: $t = ${:.1f}'.format(t)) 
        axes[0][1].grid()
        axes[0][1].legend(loc='upper right', ncol=2, fontsize=16)    
    
    for i, spec in enumerate([ispec, ospec_imo]):
        axes[1][0].plot(energies, spec.array.squeeze()[ThreeFlavor.NU_E]/1e16, 
                    label='Untransformed '+ThreeFlavor.NU_E.to_tex() if i==0 else 'Transformed '+ThreeFlavor.NU_E.to_tex(),
                    color='C0', ls='-' if i==0 else ':', lw=2,  alpha=0.7)
        axes[1][0].plot(energies, spec.array.squeeze()[ThreeFlavor.NU_MU]/1e16, 
                    label='Untransformed '+ThreeFlavor.NU_MU.to_tex() if i==0 else 'Transformed '+ThreeFlavor.NU_MU.to_tex(),
                    color='C1', ls='-' if i==0 else ':', lw=2,  alpha=0.7)

        axes[1][0].set(xlabel=r'$E$ [{}]'.format(energies.unit), title='Neutrinos in the IMO: $t = ${:.1f}'.format(t))
        axes[1][0].grid()
        axes[1][0].legend(loc='upper right', ncol=2, fontsize=16)
    
        axes[1][1].plot(energies, spec.array.squeeze()[ThreeFlavor.NU_E_BAR]/1e16, 
                    label='Untransformed '+ThreeFlavor.NU_E_BAR.to_tex() if i==0 else 'Transformed '+ThreeFlavor.NU_E_BAR.to_tex(),
                    color='C0', ls='-' if i==0 else ':', lw=2,  alpha=0.7)
        axes[1][1].plot(energies, spec.array.squeeze()[ThreeFlavor.NU_MU_BAR]/1e16, 
                    label='Untransformed '+ThreeFlavor.NU_MU_BAR.to_tex() if i==0 else 'Transformed '+ThreeFlavor.NU_MU_BAR.to_tex(),
                    color='C1', ls='-' if i==0 else ':', lw=2,  alpha=0.7)

        axes[1][1].set(xlabel=r'$E$ [{}]'.format(energies.unit), title='Antineutrinos in the IMO: $t = ${:.1f}'.format(t))
        axes[1][1].grid()
        axes[1][1].legend(loc='upper right', ncol=2, fontsize=18)    

    ax = axes[0][0]
    ax.set(ylabel=r'flux [$10^{16}$ erg$^{-1}$ cm$^{-2}$ s$^{-1}$]')
    ax = axes[1][0]
    ax.set(ylabel=r'flux [$10^{16}$ erg$^{-1}$ cm$^{-2}$ s$^{-1}$]')
    
    return fig

In [ ]:
mixpars_nmo = MixingParameters('NORMAL')
mixpars_imo = MixingParameters('INVERTED')

xform_nmo = xforms.AdiabaticMSW(mixpars_nmo)
xform_imo = xforms.AdiabaticMSW(mixpars_imo) 

fig = plot_total_flux(model, xform_nmo, xform_imo)
fig.show()
# fig.savefig('flux_adiabaticmsw.pdf')
fig = plot_spectra(model, xform_nmo, xform_imo, 100*u.ms)
fig.show()
# fig.savefig('spectra_adiabaticmsw.pdf')

In [ ]:
# set 1 degree as the mixing angle theta14
mixpars_nmo = FourFlavorMixingParameters(**MixingParameters('NORMAL'), theta14=1*u.deg)
xf_nmo = xforms.AdiabaticMSWes(mixpars_nmo)

mixpars_imo = FourFlavorMixingParameters(**MixingParameters('INVERTED'), theta14=1*u.deg)
xf_imo = xforms.AdiabaticMSWes(mixpars_imo)

fig = plot_total_flux(model, xf_nmo, xf_imo)
fig.show()
# fig.savefig('flux_adiabaticmswes.pdf')
fig = plot_spectra(model, xf_nmo, xf_imo, 100*u.ms)
fig.show()
# fig.savefig('spectra_adiabaticmswes.pdf')